# ZIP load model (test with GridCal)

## GridCal case 5 example with zip load at bus 2

In [1]:
import numpy as np

from GridCal.grid.CalculationEngine import *

np.set_printoptions(precision=4)
grid = MultiCircuit()


# Add buses
bus1 = Bus('Bus 1', vnom=20)
# bus1.is_slack = True
bus1.controlled_generators.append(ControlledGenerator('Slack Generator', voltage_module=1.0))
grid.add_bus(bus1)

bus2 = Bus('Bus 2', vnom=20)
bus2.loads.append(Load('load 2', power=0.5*complex(40, 20),
                                 impedance=1/(10.-5.j),
                                 current=np.conj(10.+5.j) / (20 * np.sqrt(3)))
                  )
grid.add_bus(bus2)

bus3 = Bus('Bus 3', vnom=20)
bus3.loads.append(Load('load 3', power=complex(25, 15)))
grid.add_bus(bus3)

bus4 = Bus('Bus 4', vnom=20)
bus4.loads.append(Load('load 4', power=complex(40, 20)))
grid.add_bus(bus4)

bus5 = Bus('Bus 5', vnom=20)
bus5.loads.append(Load('load 5', power=complex(50, 20)))
grid.add_bus(bus5)


# add branches (Lines in this case)
grid.add_branch(Branch(bus1, bus2, 'line 1-2', r=0.05, x=0.11, b=0.02))

grid.add_branch(Branch(bus1, bus3, 'line 1-3', r=0.05, x=0.11, b=0.02))

grid.add_branch(Branch(bus1, bus5, 'line 1-5', r=0.03, x=0.08, b=0.02))

grid.add_branch(Branch(bus2, bus3, 'line 2-3', r=0.04, x=0.09, b=0.02))

grid.add_branch(Branch(bus2, bus5, 'line 2-5', r=0.04, x=0.09, b=0.02))

grid.add_branch(Branch(bus3, bus4, 'line 3-4', r=0.06, x=0.13, b=0.03))

grid.add_branch(Branch(bus4, bus5, 'line 4-5', r=0.04, x=0.09, b=0.02))


grid.compile()

# print('Ybus:\n', grid.circuits[0].power_flow_input.Ybus.todense())

options = PowerFlowOptions(SolverType.NR, verbose=False, robust=False)
power_flow = PowerFlow(grid, options)
power_flow.run()

Setting the bus [0] as slack instead of pv
Setting the bus [0] as slack instead of pv
[[0, 1, 2, 3, 4]]
PowerFlow at  Grid
Setting the bus [0] as slack instead of pv


In [2]:
# print('\n\n', grid.name)
print('\t|V|:', abs(grid.power_flow_results.voltage))
print('\tVang:', np.rad2deg(np.angle(grid.power_flow_results.voltage)))
print('\t|Sbus|:', grid.power_flow_results.Sbus)
print('\t|Sbranch|:', abs(grid.power_flow_results.Sbranch))
print('\t|loading|:', abs(grid.power_flow_results.loading) * 100)
print('\terr:', grid.power_flow_results.error)
print('\tConv:', grid.power_flow_results.converged)

	|V|: [ 1.      0.9562  0.9553  0.9338  0.9538]
	Vang: [ 0.     -2.3839 -2.3523 -3.6375 -2.6778]
	|Sbus|: [ 0.00+0.j   -0.20-0.1j  -0.25-0.15j -0.40-0.2j  -0.50-0.2j ]
	|Sbranch|: [ 49.1041  49.3071  75.565    1.8908   5.3234  19.6851  24.2913]
	|loading|: [ 4910.4096  4930.7092  7556.4998   189.0838   532.3418  1968.5141  2429.1295]
	err: 1.778955111e-08
	Conv: True


## PandaPower

In [3]:
import sys
sys.path.append('/Users/jko/github/pandapower')

import pandapower as pp #import pandapower
from pandapower import converter as ppconv

# ppc case generated according to GridCal
from case5_gridcal import case5_gridcal
from pypower.api import makeYbus

ppc = case5_gridcal()

net = ppconv.from_ppc(ppc)
net.sn_kva = 1.e5

### adding zip loads to pandapower

In [4]:
net.load['const_p'] = 1.
net.load['const_i'] = 0.
net.load['const_z'] = 0.
net.load.loc[0,'const_p'] = 0.5
net.load.loc[0,'const_z'] = 0.25
net.load.loc[0,'const_i'] = 0.25

net.load

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type,const_p,const_i,const_z
0,None,1,40000.0,20000.0,NaN,1.0,True,None,0.5,0.25,0.25
1,None,2,25000.0,15000.0,NaN,1.0,True,None,1.0,0.00,0.00
2,None,3,40000.0,20000.0,NaN,1.0,True,None,1.0,0.00,0.00
3,None,4,50000.0,20000.0,NaN,1.0,True,None,1.0,0.00,0.00


In [5]:
# recycle Ybus in order compare it with GridCal's Ybus
pp.runpp(net, voltage_depend_loads=True, recycle = dict(_is_elements=False, ppc=False, Ybus=True, bfsw=False))
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-148454.305205,-65725.771826
1,0.960920,-2.141595,40000.000000,20000.000000
2,0.957955,-2.221725,25000.000000,15000.000000
3,0.936317,-3.507022,40000.000000,20000.000000
4,0.956146,-2.557031,50000.000000,20000.000000


## Pandapower vs. GridCal 

### Ybus test

In [6]:
Ybus_pp=net["_ppc"]['internal']['Ybus'].todense()
bus_ord = net["_pd2ppc_lookups"]["bus"]
Ybus_pp = Ybus_pp[bus_ord,:][:,bus_ord]

if np.allclose(Ybus_pp, grid.circuits[0].power_flow_input.Ybus.todense()):
    print("\n\tYbus PANDAPOWER = GridCal !!!")
else:
    print("Ybus ERROR")


	Ybus PANDAPOWER = GridCal !!!


### PF result test

In [7]:
if (np.allclose(net.res_bus.vm_pu, abs(grid.power_flow_results.voltage)) and
    np.allclose(np.rad2deg(np.angle(grid.power_flow_results.voltage)), net.res_bus.va_degree)):
    print("\n\t PF PANDAPOWER = GridCal !!!")
else:
    print("PF ERROR")

PF ERROR
